Data augmentation

In [1]:
import albumentations as A
import cv2
import pandas as pd
import ast
import os
from tqdm import tqdm
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import shutil
import matplotlib as plt
import yaml

In [2]:
WIDTH = 1280
HEIGHT = 720

LOCAL_PATH_ANNOTATIONS = 'data/train.csv'
LOCAL_PATH_IMG_DIR = 'data/train_images/'
LOCAL_SAVE = 'data_yolo/augmented'
LOCAL_PREFIX = 'data_yolo/'
TRAIN_IMG = 'data/images/train'
VAL_IMG = 'data/images/val'
TEST_IMG = 'data/images/test'

TRAIN_LBL = 'data/labels/train'
VAL_LBL = 'data/labels/val'
TEST_LBL = 'data/labels/test'

In [3]:
transform_flip_h = A.Compose(
    [A.HorizontalFlip(p=1)],
    bbox_params = A.BboxParams(format='yolo', label_fields = ['category_ids'])
)
transform_flip_v = A.Compose(
    [A.VerticalFlip(p=1)],
    bbox_params = A.BboxParams(format='yolo', label_fields = ['category_ids'])
)
transform_colors = A.Compose(
    [A.RandomBrightnessContrast(p=0.5),
    A.RGBShift(r_shift_limit=30, g_shift_limit=30, b_shift_limit=30, p=0.7)],
    bbox_params = A.BboxParams(format='yolo', label_fields = ['category_ids'])
)

transformations = [transform_flip_v, transform_flip_h, transform_colors]

In [4]:
def create_augmented_data(annotations_file, img_dir, save_path, transformations=transformations):
    img_labels = pd.read_csv(annotations_file)
    annotated = img_labels[img_labels['annotations'] != '[]']
    bboxes = {'id':[], 'bboxes':[]}
    i = 0
    for idx in tqdm(range(len(annotated))):
        image = cv2.imread(os.path.join(img_dir, 'video_{}'.format(annotated.iloc[idx][0]),'{}.jpg'.format(annotated.iloc[idx][2])))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        cv2.imwrite(os.path.join(save_path, 'images/im{}.jpg'.format(i)), cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
        labels = annotated.iloc[idx][-1]
        labels = ast.literal_eval(labels)
        coords = []
        for parsed_label in labels:
            x, y = parsed_label['x'], parsed_label['y']
            w, h = parsed_label['width'], parsed_label['height']
            cx, cy = min((0.5*w + x)/WIDTH, 1), min((0.5*h + y)/HEIGHT, 1)
            nw, nh = min(w/WIDTH, 1), min(h/HEIGHT, 1)
            coords.append([cx, cy, nw, nh])
        labels = [0 for _ in range(len(coords))]
        bboxes['id'].append(i)
        bboxes['bboxes'].append(coords)
        i += 1
            
        try:
            for transform in transformations:
                transformed = transform(image=image, bboxes=coords, category_ids=labels)
                img = transformed['image']
                boxes = transformed['bboxes']
                cv2.imwrite(os.path.join(save_path, 'images/im{}.jpg'.format(i)), cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
                bboxes['id'].append(i)
                bboxes['bboxes'].append(boxes)
                i += 1
        except:
            print(os.path.join(img_dir, 'video_{}'.format(annotated.iloc[idx][0]),'{}.jpg'.format(annotated.iloc[idx][2])))
    csv = pd.DataFrame(data=bboxes)
    csv.to_csv(os.path.join(save_path, 'data.csv'))
    
    

In [ ]:
create_augmented_data(LOCAL_PATH_ANNOTATIONS, LOCAL_PATH_IMG_DIR, LOCAL_SAVE)

  4%|▍         | 194/4919 [00:37<16:26,  4.79it/s]

Dataset preparation - yolo

In [ ]:
class StarfishDataset(Dataset):
    def __init__(self,
                 annotations_file='data_yolo/augmented/data.csv',
                 img_dir='data_yolo/augmented/images'
                 ):
        self.img_labels = pd.read_csv(annotations_file)
        self.annotated = self.img_labels[self.img_labels['bboxes'] != '[]']  # get only annotated frames
        self.img_dir = img_dir

    def __len__(self):
        return len(self.annotated)

    def __getitem__(self, idx):
        image = os.path.join(self.img_dir, 'im{}.jpg'.format(self.annotated.iloc[idx][0]))
        coords = self.annotated.iloc[idx][-1]
        coords = np.array(ast.literal_eval(coords))
        labels = np.array([0 for _ in range(len(coords))])
        labels = np.expand_dims(labels, axis=0)
        labels = np.concatenate((labels.T, coords), axis=1)
        boxes = np.array(labels)
        return image, boxes

In [ ]:
dataset = StarfishDataset()
print(len(dataset))

In [ ]:
train_size = 1500
val_size = 10000
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(dataset, (train_size, val_size, test_size))

print('Train dataset: {} instances, validation dataset: {}, test dataset: {}'.format(len(train_dataset), len(val_dataset), len(test_dataset)))

In [ ]:
def prepare_dataset(dataset, path_img, path_lbl):
    i = 0
    for (image, label) in dataset:
        file_image = os.path.join(path_img ,'im{}.jpg'.format(i))
        file_label = os.path.join(path_lbl ,'im{}.txt'.format(i))
        shutil.copyfile(image, file_image)
        np.savetxt(file_label, label, fmt='%i %.4f %.4f %.4f %.4f')
        i += 1

In [ ]:
prepare_dataset(train_dataset, LOCAL_PREFIX+TRAIN_IMG, LOCAL_PREFIX+TRAIN_LBL)

In [ ]:
prepare_dataset(val_dataset, LOCAL_PREFIX+VAL_IMG, LOCAL_PREFIX+VAL_LBL)

In [ ]:
prepare_dataset(test_dataset, LOCAL_PREFIX+TEST_IMG, LOCAL_PREFIX+TEST_LBL)